In [ ]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.models import Sequential, Model
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow.keras.activations as activations
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive
import os

import numpy as np

In [ ]:
drive.mount('/gdrive', force_remount=True)

In [ ]:
%cd /gdrive/My\ Drive/IU_MSDS/2021Fall_E533_DL/DLS\ Final\ Project\ /CUB_200_Numpy/
os.listdir()

In [ ]:
desired_image_shape = (64, 64)
np_save_file_name = 'CUB_200_' + str(desired_image_shape[0]) + '_by_' + str(desired_image_shape[1]) + '.npy'
X_train = np.load(np_save_file_name)
X_train = X_train.astype('float32')
X_train = X_train[:-12]
np.random.shuffle(X_train)
print(f'\n{X_train.shape[0]} Images of the size {X_train.shape[1]}x{X_train.shape[2]} with {X_train.shape[3]} channels.')

In [ ]:
plt.figure(1, figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(X_train[i] * 0.5 + 0.5)
    plt.axis('off')
plt.show()

In [ ]:
class GAN():
    def __init__(self):
        
        self.img_rows, self.img_cols, self.channels, self.latent_dim = X_train.shape[1], X_train.shape[2], X_train.shape[3], 100
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer1 = Adam(learning_rate=0.00015, beta_1=0.5)
        optimizer2 = Adam(learning_rate=0.00015, beta_1=0.5)
        
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer1, metrics=['accuracy'])
        
        self.generator = self.build_generator()

        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)
        validity = self.discriminator(img)
        self.combined = Model(z, validity)

        self.discriminator.trainable = False
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer2, metrics=['accuracy'])
        
        
    def build_generator(self):

        model = Sequential()
        model.add(Input(shape=(self.latent_dim,)))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.img_rows * self.img_cols * self.channels, activation='tanh'))
        model.add(Reshape(self.img_shape))
        
        return model
    
    
    def build_discriminator(self):

        model=Sequential()
        model.add(Input(shape=self.img_shape))
        model.add(Flatten())
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.3))
        model.add(Dense(64))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.3))
        model.add(Dense(1, activation='sigmoid'))
        
        return model
    
    
    def train(self, X_train, epochs=30, batch_size=64, sample_interval=5):
                
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))

        d_loss = []
        g_loss = []

        for epoch in range(epochs):
            
            d_loss_epoch = []
            g_loss_epoch = []
            
            for ite in range(0, X_train.shape[0], batch_size):
                
                real_images = X_train[ite:ite+batch_size]
                
                noise1 = np.random.normal(0, 1, (batch_size, self.latent_dim))

                fake_images = self.generator.predict(noise1)
                
                all_images = np.vstack([real_images, fake_images])
                all_labels = np.vstack([real_labels, fake_labels])
                
                
                shuffler = np.random.permutation(all_images.shape[0])
                
                all_images = all_images[shuffler]
                all_labels = all_labels[shuffler]

                d_loss_epoch.append(self.discriminator.train_on_batch(all_images, all_labels)[0])
                
                noise2 = np.random.normal(0, 1, (batch_size, self.latent_dim))
                
                g_loss_epoch.append(self.combined.train_on_batch(noise2, real_labels)[0])

                
            g_loss.append(sum(g_loss_epoch)/len(g_loss_epoch))
            d_loss.append(sum(d_loss_epoch)/len(d_loss_epoch))
            
            if epoch % sample_interval == 0:
                
                print(f'Epoch {epoch} - D loss: {d_loss[-1]:.5f}, G_loss: {g_loss[-1]:.5f}')
                self.sample_images(epoch)
                
        return d_loss, g_loss
        
    def sample_images(self, epoch):
        r, c = 3, 3

        fig, axs = plt.subplots(r, c, figsize=(6, 6))
        for i in range(r):
            for j in range(c):
                noise = np.random.normal(0, 1, (1, self.latent_dim))
                gen_img = self.generator.predict(noise)[0]
                axs[i,j].imshow(gen_img * 0.5 + 0.5)
                axs[i,j].axis('off')
        plt.show()

In [ ]:
g = GAN()

In [ ]:
d_loss, g_loss = g.train(X_train, 200, 32, 1)

In [ ]:
%cd /gdrive/My\ Drive/IU_MSDS/2021Fall_E533_DL/DLS\ Final\ Project\ /Project_Notebooks/Project_Models/

In [ ]:
g.generator.save_weights('generator_dense_64.h5')
g.discriminator.save_weights('discriminator_dense_64.h5')
g.combined.save_weights('combined_dense_64.h5')

In [ ]:
g_test = GAN()
gen = g_test.generator
gen.load_weights('generator_dense_64.h5')

r, c = 3, 3

fig, axs = plt.subplots(r, c, figsize=(6, 6))
for i in range(r):
    for j in range(c):
        noise = np.random.normal(0, 1, (1, 100))
        gen_img = gen.predict(noise)[0]
        axs[i,j].imshow(gen_img * 0.5 + 0.5)
        axs[i,j].axis('off')
plt.show()


noise = np.random.normal(0, 1, (1, 100))
gen_img =  gen.predict(noise)[0] * 0.5 + 0.5
plt.imshow(gen_img)
plt.show()